In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

rawdata=pd.read_csv('./data/mobike_shanghai_sample_updated.csv')
rawdata.head()

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track
0,78387,158357,10080,2016-08-20 06:57,121.348,31.389,2016-08-20 07:04,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1..."
1,891333,92776,6605,2016-08-29 19:09,121.508,31.279,2016-08-29 19:31,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1..."
2,1106623,152045,8876,2016-08-13 16:17,121.383,31.254,2016-08-13 16:36,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1..."
3,1389484,196259,10648,2016-08-23 21:34,121.484,31.320,2016-08-23 21:43,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1..."
4,188537,78208,11735,2016-08-16 07:32,121.407,31.292,2016-08-16 07:41,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1..."


In [15]:
rawdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   orderid           102361 non-null  int64  
 1   bikeid            102361 non-null  int64  
 2   userid            102361 non-null  int64  
 3   start_time        102361 non-null  object 
 4   start_location_x  102361 non-null  float64
 5   start_location_y  102361 non-null  float64
 6   end_time          102361 non-null  object 
 7   end_location_x    102361 non-null  float64
 8   end_location_y    102361 non-null  float64
 9   track             102361 non-null  object 
dtypes: float64(4), int64(3), object(3)
memory usage: 7.8+ MB


In [16]:
# 复制数据集，并更改各列的数据类型
mobike = rawdata.copy()
tobestr = ['orderid', 'bikeid', 'userid']
mobike[tobestr] = mobike[tobestr].astype('str')
mobike['start_time'] = pd.to_datetime(mobike['start_time'])
mobike['end_time'] = pd.to_datetime(mobike['end_time'])
mobike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   orderid           102361 non-null  object        
 1   bikeid            102361 non-null  object        
 2   userid            102361 non-null  object        
 3   start_time        102361 non-null  datetime64[ns]
 4   start_location_x  102361 non-null  float64       
 5   start_location_y  102361 non-null  float64       
 6   end_time          102361 non-null  datetime64[ns]
 7   end_location_x    102361 non-null  float64       
 8   end_location_y    102361 non-null  float64       
 9   track             102361 non-null  object        
dtypes: datetime64[ns](2), float64(4), object(4)
memory usage: 7.8+ MB


#### 新增骑行时长duration等列（通过起始时间计算），并通过字符串拆分将骑行时长单位统一为minutes（命名为ttl_min列）

In [17]:
mobike['duration'] = mobike.end_time - mobike.start_time
mobike['dur_day'] = mobike.duration.apply(lambda x: str(x).split(' ')[0])
mobike['dur_hr'] = mobike.duration.apply(lambda x: str(x).split(' ')[-1][:2])
mobike['dur_min'] = mobike.duration.apply(lambda x: str(x).split(':')[-2])
mobike['dur_sec'] = mobike.duration.apply(lambda x: str(x).split(':')[-1])
tobeint = ['dur_day', 'dur_hr', 'dur_min', 'dur_sec']
mobike[tobeint] = mobike[tobeint].astype('int')
mobike['ttl_min'] = mobike.dur_day * 24 * 60 + mobike.dur_hr * 60 + mobike.dur_min + mobike.dur_sec / 60

#### 新增dayid和daytype列，获取每条记录是星期几，并根据工作日和周末进行分类

In [18]:
mobike['dayid'] = mobike.start_time.apply(lambda x: x.isoweekday())
mobike['daytype'] = mobike.dayid.apply(lambda x: 'weekends' if x == 6 or x == 7 else 'weekdays')
mobike

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track,duration,dur_day,dur_hr,dur_min,dur_sec,ttl_min,dayid,daytype
0,78387,158357,10080,2016-08-20 06:57:00,121.348,31.389,2016-08-20 07:04:00,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1...",0 days 00:07:00,0,0,7,0,7.0,6,weekends
1,891333,92776,6605,2016-08-29 19:09:00,121.508,31.279,2016-08-29 19:31:00,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1...",0 days 00:22:00,0,0,22,0,22.0,1,weekdays
2,1106623,152045,8876,2016-08-13 16:17:00,121.383,31.254,2016-08-13 16:36:00,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1...",0 days 00:19:00,0,0,19,0,19.0,6,weekends
3,1389484,196259,10648,2016-08-23 21:34:00,121.484,31.320,2016-08-23 21:43:00,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1...",0 days 00:09:00,0,0,9,0,9.0,2,weekdays
4,188537,78208,11735,2016-08-16 07:32:00,121.407,31.292,2016-08-16 07:41:00,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1...",0 days 00:09:00,0,0,9,0,9.0,2,weekdays
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102356,1479550,214335,1423,2016-08-28 18:03:00,121.478,31.297,2016-08-28 18:09:00,121.481,31.304,"121.478,31.297#121.479,31.297#121.480,31.297#1...",0 days 00:06:00,0,0,6,0,6.0,7,weekends
102357,1478273,160487,3067,2016-08-14 20:22:00,121.320,31.238,2016-08-14 20:28:00,121.312,31.235,"121.312,31.235#121.313,31.235#121.313,31.236#1...",0 days 00:06:00,0,0,6,0,6.0,7,weekends
102358,367733,179530,12746,2016-08-27 09:54:00,121.391,31.307,2016-08-27 10:06:00,121.398,31.306,"121.390,31.307#121.391,31.307#121.392,31.307#1...",0 days 00:12:00,0,0,12,0,12.0,6,weekends
102359,64915,167419,837,2016-08-20 06:04:00,121.515,31.269,2016-08-20 06:10:00,121.510,31.272,"121.510,31.272#121.510,31.273#121.511,31.273#1...",0 days 00:06:00,0,0,6,0,6.0,6,weekends


新增hourid和hourtype列，获取每条记录是几点开始的，并根据早晚高峰和平峰时段进行分类

In [19]:
mobike['hourid'] = mobike.start_time.apply(lambda x: x.utctimetuple().tm_hour)
mobike['hourtype'] = mobike.hourid.apply(lambda x: 'rush hours' if (x >= 7 and x <= 8) or (x >= 17 and x <= 20) else 'non-rush hours')
mobike

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track,duration,dur_day,dur_hr,dur_min,dur_sec,ttl_min,dayid,daytype,hourid,hourtype
0,78387,158357,10080,2016-08-20 06:57:00,121.348,31.389,2016-08-20 07:04:00,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1...",0 days 00:07:00,0,0,7,0,7.0,6,weekends,6,non-rush hours
1,891333,92776,6605,2016-08-29 19:09:00,121.508,31.279,2016-08-29 19:31:00,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1...",0 days 00:22:00,0,0,22,0,22.0,1,weekdays,19,rush hours
2,1106623,152045,8876,2016-08-13 16:17:00,121.383,31.254,2016-08-13 16:36:00,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1...",0 days 00:19:00,0,0,19,0,19.0,6,weekends,16,non-rush hours
3,1389484,196259,10648,2016-08-23 21:34:00,121.484,31.320,2016-08-23 21:43:00,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1...",0 days 00:09:00,0,0,9,0,9.0,2,weekdays,21,non-rush hours
4,188537,78208,11735,2016-08-16 07:32:00,121.407,31.292,2016-08-16 07:41:00,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1...",0 days 00:09:00,0,0,9,0,9.0,2,weekdays,7,rush hours
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102356,1479550,214335,1423,2016-08-28 18:03:00,121.478,31.297,2016-08-28 18:09:00,121.481,31.304,"121.478,31.297#121.479,31.297#121.480,31.297#1...",0 days 00:06:00,0,0,6,0,6.0,7,weekends,18,rush hours
102357,1478273,160487,3067,2016-08-14 20:22:00,121.320,31.238,2016-08-14 20:28:00,121.312,31.235,"121.312,31.235#121.313,31.235#121.313,31.236#1...",0 days 00:06:00,0,0,6,0,6.0,7,weekends,20,rush hours
102358,367733,179530,12746,2016-08-27 09:54:00,121.391,31.307,2016-08-27 10:06:00,121.398,31.306,"121.390,31.307#121.391,31.307#121.392,31.307#1...",0 days 00:12:00,0,0,12,0,12.0,6,weekends,9,non-rush hours
102359,64915,167419,837,2016-08-20 06:04:00,121.515,31.269,2016-08-20 06:10:00,121.510,31.272,"121.510,31.272#121.510,31.273#121.511,31.273#1...",0 days 00:06:00,0,0,6,0,6.0,6,weekends,6,non-rush hours


#### 新增地理位置信息相关的列

新增distance和distocenter列，获取每条记录骑行起始点的直线距离和距离上海中心点的距离（km）

In [20]:
#按每条记录的起始位置，作为发

新增hourid和hourtype列，获取每条记录是几点开始的，并根据早晚高峰和平峰时段进行分类

In [ ]:
mobike['hourid'] = mobike.start_time.apply(lambda x: x.utctimetuple().tm_hour)
mobike['hourtype'] = mobike.hourid.apply(lambda x: 'rush hours' if (x >= 7 and x <= 8) or (x >= 17 and x <= 20) else 'non-rush hours')
mobike

#### 新增地理位置信息相关的列

新增distance和distocenter列，获取每条记录骑行起始点的直线距离和距离上海中心点的距离（km）

In [ ]:
# 按每条记录的起点位置，作为发起订单所处位置的数据依据
from math import radians, cos, sin, asin, sqrt

# 自定义函数，通过两点的经纬度计算两点之间的直线距离
def geodistance(lng1, lat1, lng2, lat2):
    lng1_r, lat1_r, lng2_r, lat2_r = map(radians, [lng1, lat1, lng2, lat2]) # 经纬度转换成弧度
    dlon = lng1_r - lng2_r
    dlat = lat1_r - lat2_r
    dis = sin(dlat/2)**2 + cos(lat1_r) * cos(lat2_r) * sin(dlon/2)**2
    distance = 2 * asin(sqrt(dis)) * 6371 * 1000 # 地球平均半径为6371km
    distance = round(distance/1000,3)
    return distance

# 自定义函数，通过调用geodistance获取每条记录骑行始末点和起点距中心点的直线距离
def get_dis(item):
    item['distance'] = geodistance(item['start_location_x'], item['start_location_y'],
                                   item['end_location_x'], item['end_location_y'])    # 计算骑行始末点经纬度的直线距离
    # 国际饭店一般被认为是上海地理中心坐标点，计算骑行起始点经纬度和国际饭店经纬度的直线距离
    item['distocenter'] = geodistance(item['start_location_x'], item['start_location_y'], 121.471632, 31.233705)
    return item

mobike['distance'] = np.nan
mobike['distocenter'] = np.nan
mobike['ring_stage'] = np.nan
mobike = mobike.apply(get_dis, axis=1)
mobike

新增ring_stage列，对四级环线范围进行分类

In [ ]:
#自定义函数，按照每条记录距离上海中心点（国际饭店）的距离，根据上海内中外环线进行粗略的地理位置分类
def get_ring(item):
    if item['distocenter'] <=10:
        item['ring_stage']

#自定义函数，按照每条记录距离上海中心点（国际饭店）的距离，根据上海内中外环线进行粗略的地理位置分类
def get_ring(item):
    if item['distocenter'] <=10:
        item['ring_stage']

In [22]:
mobike['hourid'] = mobike.start_time.apply(lambda x: x.utctimetuple().tm_hour)
mobike['hourtype'] = mobike.hourid.apply(lambda x: 'rush hours' if (x >= 7 and x <= 8) or (x >= 17 and x <= 20) else 'non-rush hours')
mobike

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track,duration,dur_day,dur_hr,dur_min,dur_sec,ttl_min,dayid,daytype,hourid,hourtype
0,78387,158357,10080,2016-08-20 06:57:00,121.348,31.389,2016-08-20 07:04:00,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1...",0 days 00:07:00,0,0,7,0,7.0,6,weekends,6,non-rush hours
1,891333,92776,6605,2016-08-29 19:09:00,121.508,31.279,2016-08-29 19:31:00,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1...",0 days 00:22:00,0,0,22,0,22.0,1,weekdays,19,rush hours
2,1106623,152045,8876,2016-08-13 16:17:00,121.383,31.254,2016-08-13 16:36:00,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1...",0 days 00:19:00,0,0,19,0,19.0,6,weekends,16,non-rush hours
3,1389484,196259,10648,2016-08-23 21:34:00,121.484,31.320,2016-08-23 21:43:00,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1...",0 days 00:09:00,0,0,9,0,9.0,2,weekdays,21,non-rush hours
4,188537,78208,11735,2016-08-16 07:32:00,121.407,31.292,2016-08-16 07:41:00,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1...",0 days 00:09:00,0,0,9,0,9.0,2,weekdays,7,rush hours
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102356,1479550,214335,1423,2016-08-28 18:03:00,121.478,31.297,2016-08-28 18:09:00,121.481,31.304,"121.478,31.297#121.479,31.297#121.480,31.297#1...",0 days 00:06:00,0,0,6,0,6.0,7,weekends,18,rush hours
102357,1478273,160487,3067,2016-08-14 20:22:00,121.320,31.238,2016-08-14 20:28:00,121.312,31.235,"121.312,31.235#121.313,31.235#121.313,31.236#1...",0 days 00:06:00,0,0,6,0,6.0,7,weekends,20,rush hours
102358,367733,179530,12746,2016-08-27 09:54:00,121.391,31.307,2016-08-27 10:06:00,121.398,31.306,"121.390,31.307#121.391,31.307#121.392,31.307#1...",0 days 00:12:00,0,0,12,0,12.0,6,weekends,9,non-rush hours
102359,64915,167419,837,2016-08-20 06:04:00,121.515,31.269,2016-08-20 06:10:00,121.510,31.272,"121.510,31.272#121.510,31.273#121.511,31.273#1...",0 days 00:06:00,0,0,6,0,6.0,6,weekends,6,non-rush hours


#### 新增地理位置信息相关的列

新增distance和distocenter列，获取每条记录骑行起始点的直线距离和距离上海中心点的距离（km）

In [27]:
# 按每条记录的起点位置，作为发起订单所处位置的数据依据
from math import radians, cos, sin, asin, sqrt

# 自定义函数，通过两点的经纬度计算两点之间的直线距离
def geodistance(lng1, lat1, lng2, lat2):
    lng1_r, lat1_r, lng2_r, lat2_r = map(radians, [lng1, lat1, lng2, lat2]) # 经纬度转换成弧度
    dlon = lng1_r - lng2_r
    dlat = lat1_r - lat2_r
    dis = sin(dlat/2)**2 + cos(lat1_r) * cos(lat2_r) * sin(dlon/2)**2
    distance = 2 * asin(sqrt(dis)) * 6371 * 1000 # 地球平均半径为6371km
    distance = round(distance/1000,3)
    return distance

# 自定义函数，通过调用geodistance获取每条记录骑行始末点和起点距中心点的直线距离
def get_dis(item):
    item['distance'] = geodistance(item['start_location_x'], item['start_location_y'],
                                   item['end_location_x'], item['end_location_y'])    # 计算骑行始末点经纬度的直线距离
    # 国际饭店一般被认为是上海地理中心坐标点，计算骑行起始点经纬度和国际饭店经纬度的直线距离
    item['distocenter'] = geodistance(item['start_location_x'], item['start_location_y'], 121.471632, 31.233705)
    return item

mobike['distance'] = np.nan
mobike['distocenter'] = np.nan
mobike['ring_stage'] = np.nan
mobike = mobike.apply(get_dis, axis=1)
mobike

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track,...,dur_min,dur_sec,ttl_min,dayid,daytype,hourid,hourtype,distance,distocenter,ring_stage
0,78387,158357,10080,2016-08-20 06:57:00,121.348,31.389,2016-08-20 07:04:00,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1...",...,7,0,7.0,6,weekends,6,non-rush hours,0.862,20.884,NaN
1,891333,92776,6605,2016-08-29 19:09:00,121.508,31.279,2016-08-29 19:31:00,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1...",...,22,0,22.0,1,weekdays,19,rush hours,2.013,6.109,NaN
2,1106623,152045,8876,2016-08-13 16:17:00,121.383,31.254,2016-08-13 16:36:00,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1...",...,19,0,19.0,6,weekends,16,non-rush hours,2.195,8.723,NaN
3,1389484,196259,10648,2016-08-23 21:34:00,121.484,31.320,2016-08-23 21:43:00,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1...",...,9,0,9.0,2,weekdays,21,non-rush hours,1.354,9.667,NaN
4,188537,78208,11735,2016-08-16 07:32:00,121.407,31.292,2016-08-16 07:41:00,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1...",...,9,0,9.0,2,weekdays,7,rush hours,1.136,8.931,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102356,1479550,214335,1423,2016-08-28 18:03:00,121.478,31.297,2016-08-28 18:09:00,121.481,31.304,"121.478,31.297#121.479,31.297#121.480,31.297#1...",...,6,0,6.0,7,weekends,18,rush hours,0.829,7.064,NaN
102357,1478273,160487,3067,2016-08-14 20:22:00,121.320,31.238,2016-08-14 20:28:00,121.312,31.235,"121.312,31.235#121.313,31.235#121.313,31.236#1...",...,6,0,6.0,7,weekends,20,rush hours,0.831,14.424,NaN
102358,367733,179530,12746,2016-08-27 09:54:00,121.391,31.307,2016-08-27 10:06:00,121.398,31.306,"121.390,31.307#121.391,31.307#121.392,31.307#1...",...,12,0,12.0,6,weekends,9,non-rush hours,0.674,11.187,NaN
102359,64915,167419,837,2016-08-20 06:04:00,121.515,31.269,2016-08-20 06:10:00,121.510,31.272,"121.510,31.272#121.510,31.273#121.511,31.273#1...",...,6,0,6.0,6,weekends,6,non-rush hours,0.581,5.692,NaN


新增ring_stage列，对四级环线范围进行分类

In [ ]:
#自定义函数，按照每条记录距离上海中心点（国际饭店）的距离，根据上海内中外环线进行粗略的地理位置分类
def get_ring(item):
    if item['distocenter'] <=10:
        item['ring_stage']